In [ ]:
%pip install jira
%pip install jira-worklog-manager-domain

In [ ]:
from jira import JIRA

from jira_worklog_manager_domain.jira_dtos import (
    JiraInfo,
    UserWorklogItemDto,
    UserWorklogsInfo,
)
from jira_worklog_manager_domain.jira_helpers import (
    get_total_task_hours,
    get_user_worklogs_info,
    get_worklog_date_range,
    map_trimmered_issues_worklogs,
)


In [ ]:
# NOTE: a sensitive data
user_login = "JIRA LOGIN"
user_jira_password = "JIRA PASSWORD"
jira_info = JiraInfo(
    jira_server_address="https://jira.com/",
    basic_auth=(user_login, user_jira_password),
    user_login=user_login,
)


In [ ]:
auth_jira = JIRA(
    basic_auth=jira_info.basic_auth, options={"server": jira_info.jira_server_address}
)


In [ ]:
start_worklog_date = "2022-10-14"
end_worklog_date = "2022-11-13"
jql = f'worklogAuthor in ("{jira_info.user_login}") and worklogDate >= {start_worklog_date} and worklogDate < {end_worklog_date}'
found_issues = auth_jira.search_issues(jql)


In [ ]:
found_issues = map_trimmered_issues_worklogs(found_issues, auth_jira)


In [ ]:
worklogs_data_range = get_worklog_date_range(start_worklog_date, end_worklog_date)
user_worklog_info = UserWorklogsInfo(
    issues=found_issues, worklogs_data_range=worklogs_data_range, user_login=user_login
)
user_worklogs = get_user_worklogs_info(user_worklog_info)
json_user_worklogs = UserWorklogItemDto.schema().dumps(user_worklogs, many=True)
total_task_hours = get_total_task_hours(user_worklogs)

print(f"total task hours: {total_task_hours}")
print()
print(json_user_worklogs)
